In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-load.v3/fact/granularity=daily/



In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select count(1) as cnt 
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_1
    where date = '2020-04-11'  and isunique='t' ) as prod
\$proxy\$) tbl (cnt bigint) ;

select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select count(1) as cnt 
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_0
    where date = '2020-04-11'  and isunique='t' ) as prod
\$proxy\$) tbl (cnt bigint) 

EOF


# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# select feed_id, sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
#     select feed_id, sum(estimate) as cnt 
#     from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_1
#     where date = '2019-12-05'  and  feed_id in (0,1,2,100,101,102) and isunique='t' ) as prod group by feed_id
# \$proxy\$) tbl (feed_id smallint, cnt bigint) group by feed_id


# EOF



# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# select  app_id,feed_id,store_id,estimate,category_id ,device_id, rank, date from plproxy.execute_select_nestloop(\$proxy\$ 
# select  distinct app_id,feed_id,store_id,estimate,category_id ,device_id, rank, date
#     from aa.app_store_daily_estimate_1
#     where 
#       date between '2020-05-10' and '2020-05-10' and feed_id=0  limit 500 \$proxy\$) tbl 
#       (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT, category_id INT, device_id SMALLINT, rank INT, date Date ) order by category_id  asc, date desc limit 500 ;

# EOF


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select * from store.store_est_fact_v6 where date='2020-05-10' and app_id=331975235 and country_code='PK' 
EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select * from store.store_est_t_w_fact_v6 where est_free_app_download=0 limit 5
EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select * from store.store_est_t_w_fact_v6 where est_paid_download != 0 limit 5
EOF

In [0]:


spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-02-01/platform=android").createOrReplaceTempView("t")
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-02-01/device_code=android-all/").createOrReplaceTempView("q")
spark.sql("select * from t where feed=0 and id=20600009210972 and store_id=40").show()
# spark.sql("select sum(free_app_download) from q where free_app_download is not null").show()

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h   -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select device_code, sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue),  sum(est_organic_download),  sum(est_paid_download) from store.store_est_fact_v6 where date='2020-01-01' group by device_code


In [0]:

ios_feed={1:"0,1,2,100,101,102"}
android_feed={0:"0,1,2"}

print type(ios_feed.keys()[0])

if ios_feed.keys() == 1:
    store_id = 1
else:
    store_id = 1001



In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row


start = "2014-11-12"
end = "2014-11-13"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append((str(real_date1 + datetime.timedelta(days))))
dates.sort(reverse=True)


ios_feed={1:"0,1,2,100,101,102"}
android_feed={0:"0,1,2"}

def plproxy_row(date, device_feed_dict):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )


    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }
    
    
    settings = build_db_settings(urn, "DAILY_EST")


    if device_feed_dict.keys()[0] == 1:
        store_id = 1
    else:
        store_id = 1001

    sql = '''select feed_id, Cast(sum(cnt) as int) as metric_sum from plproxy.execute_select_nestloop($$
    select feed_id, sum(estimate)  as cnt
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_{}
        where date = '{}'  and  feed_id in ({}) and isunique='t' and store_id not in ({}) ) as prod group by feed_id 
        $$) tbl (feed_id smallint, cnt bigint) group by feed_id '''.format(
     device_feed_dict.keys()[0], date, ','.join(device_feed_dict.values()), store_id
    )

    print sql

    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [ Row(feed_id=r[0], metric_sum=r[1]) for r in rows ]



def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select device_code, cast(sum(est_free_app_download) as int) as est_free_app_download, cast(sum(est_paid_app_download) as int) as est_paid_app_download, cast(sum(est_revenue) as int) as est_revenue from store.store_est_fact_v6 where date='{}' group by device_code".format(date )
        db_data = query(citus_dsn_, sql)
        return db_data

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    
    result = get_data_in_citus(date)
    return [ Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2], sum_est_revenue=r[3]) for r in result ]


for d in dates:
    print d
    r1 = plproxy_row(d,ios_feed)
    r2 = plproxy_row(d,android_feed)
    citus_reseult = citus_row(d)


    d1 = generate_plploxy_result(r1,r2)
    d2 = generate_citus_result(citus_reseult)

    # d1.show()
    # d2.show()

    
    d1.createOrReplaceTempView("plploxy_r")
    d2.createOrReplaceTempView("citus_r")
    spark.sql("select * from plploxy_r except all select * from citus_r").show()
    spark.sql("select * from citus_r except all select * from plploxy_r").show()


# 2020-02-23
# +-----------+-------------------------+-------------------------+---------------+
# |device_code|sum_est_free_app_download|sum_est_paid_app_download|sum_est_revenue|
# +-----------+-------------------------+-------------------------+---------------+
# |  ios-phone|                175958534|                  1957879|      192484890|
# +-----------+-------------------------+-------------------------+---------------+

# +-----------+-------------------------+-------------------------+---------------+
# |device_code|sum_est_free_app_download|sum_est_paid_app_download|sum_est_revenue|
# +-----------+-------------------------+-------------------------+---------------+
# |  ios-phone|                175936729|                  1957879|      192484890|
# +-----------+-------------------------+-------------------------+---------------+


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row


start = "2020-02-01"
end = "2020-03-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append((str(real_date1 + datetime.timedelta(days))))
dates.sort(reverse=True)


ios_feed={1:"0,1,2,100,101,102"}
android_feed={0:"0,1,2"}

def plproxy_row(date, device_feed_dict):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )


    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }
    
    
    settings = build_db_settings(urn, "DAILY_EST")

    sql = '''select feed_id, Cast(sum(cnt) as int) as metric_sum from plproxy.execute_select_nestloop($$
    select feed_id, sum(estimate)  as cnt
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_{}
        where date = '{}'  and  feed_id in ({}) ) as prod group by feed_id 
        $$) tbl (feed_id smallint, cnt bigint) group by feed_id '''.format(
     device_feed_dict.keys()[0], date, ','.join(device_feed_dict.values())
    )



    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [ Row(feed_id=r[0], metric_sum=r[1]) for r in rows ]



def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select device_code, cast(sum(est_free_app_download) as int) as est_free_app_download, cast(sum(est_paid_app_download) as int) as est_paid_app_download, cast(sum(est_revenue) as int) as est_revenue from store.store_est_category_fact_v6 where date='{}' group by device_code".format(date )
        db_data = query(citus_dsn_, sql)
        return db_data

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    
    result = get_data_in_citus(date)
    return [ Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2], sum_est_revenue=r[3]) for r in result ]


for d in dates:
    print d
    r1 = plproxy_row(d,ios_feed)
    r2 = plproxy_row(d,android_feed)
    citus_reseult = citus_row(d)


    d1 = generate_plploxy_result(r1,r2)
    d2 = generate_citus_result(citus_reseult)

    # d1.show()
    # d2.show()

    
    d1.createOrReplaceTempView("plploxy_r")
    d2.createOrReplaceTempView("citus_r")
    spark.sql("select * from plploxy_r except all select * from citus_r").show()
    spark.sql("select * from citus_r except all select * from plploxy_r").show()


# 2020-02-23
# +-----------+-------------------------+-------------------------+---------------+
# |device_code|sum_est_free_app_download|sum_est_paid_app_download|sum_est_revenue|
# +-----------+-------------------------+-------------------------+---------------+
# |  ios-phone|                175958534|                  1957879|      192484890|
# +-----------+-------------------------+-------------------------+---------------+

# +-----------+-------------------------+-------------------------+---------------+
# |device_code|sum_est_free_app_download|sum_est_paid_app_download|sum_est_revenue|
# +-----------+-------------------------+-------------------------+---------------+
# |  ios-phone|                175936729|                  1957879|      192484890|
# +-----------+-------------------------+-------------------------+---------------+


In [0]:

from pyspark.sql.types import *
def generate_plploxy_result(r1, r2):
    schema = StructType([
    StructField("feed_id", StringType(), True),
    StructField("metric_sum", IntegerType(), True)])
    df1 = spark.createDataFrame(r1,schema)
    df1.createOrReplaceTempView("plploxy_ios")
    
    df2 = spark.createDataFrame(r2,schema)
    df2.createOrReplaceTempView("plploxy_android")
    
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "sum_est_free_app_download"
    WHEN feed_id='100' THEN "sum_est_paid_app_download"
    WHEN feed_id='102' THEN "sum_est_revenue"
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_ios
    ''').createOrReplaceTempView("plploxy_metric")
    
    
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_android
    ''').createOrReplaceTempView("plploxy_metric_android")
    
    
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "ios-tablet"
    WHEN feed_id='100' THEN "ios-tablet"
    WHEN feed_id='102' THEN "ios-tablet"
    WHEN feed_id='0' THEN "ios-phone"
    WHEN feed_id='1' THEN "ios-phone"
    WHEN feed_id='2' THEN "ios-phone"
    END AS device_code from plploxy_metric
    ''').createOrReplaceTempView("plploxy_metric_device_code")
    
    
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "android-all"
    WHEN feed_id='1' THEN "android-all"
    WHEN feed_id='2' THEN "android-all"
    END AS device_code from plploxy_metric_android
    ''').createOrReplaceTempView("plploxy_metric_device_code_android")
    
    
    spark.sql('''
    SELECT * FROM plploxy_metric_device_code_android
    UNION ALL
    SELECT * FROM plploxy_metric_device_code
    ''').createOrReplaceTempView("all_device")
    
    spark.sql('''
    SELECT 
        device_code, sum_est_free_app_download , sum_est_paid_app_download , sum_est_revenue
    FROM
          all_device
     PIVOT (
        max(metric_sum) 
        FOR metric IN ('sum_est_free_app_download','sum_est_paid_app_download', 'sum_est_revenue')
      )
    ''').createOrReplaceTempView("after_pivot")
    
    spark.sql('''
    SELECT device_code, sum_est_free_app_download FROM after_pivot
    WHERE sum_est_free_app_download is not null
    ''').createOrReplaceTempView("after_pivot_1")

    spark.sql('''
    SELECT device_code, sum_est_paid_app_download FROM after_pivot
    WHERE sum_est_paid_app_download is not null
    ''').createOrReplaceTempView("after_pivot_2")


    spark.sql('''
    SELECT device_code, sum_est_revenue FROM after_pivot
    WHERE sum_est_revenue is not null
    ''').createOrReplaceTempView("after_pivot_3")

    return spark.sql('''
    SELECT c.device_code, c.sum_est_free_app_download, c.sum_est_paid_app_download, d.sum_est_revenue
    FROM (
    SELECT a.device_code, a.sum_est_free_app_download, b.sum_est_paid_app_download FROM after_pivot_1 a
    JOIN after_pivot_2 b
    ON a.device_code=b.device_code
    ) AS c
    JOIN after_pivot_3 d
    on c.device_code=d.device_code
    order by device_code desc
    ''')

    


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row

def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select device_code, cast(sum(est_free_app_download) as int) as est_free_app_download, cast(sum(est_paid_app_download) as int) as est_paid_app_download, cast(sum(est_revenue) as int) as est_revenue from store.store_est_fact_v6 where date='{}' group by device_code".format(date )
        db_data = query(citus_dsn_, sql)
        return db_data

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    
    result = get_data_in_citus(date)
    return [ Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2], sum_est_revenue=r[3]) for r in result ]


citus_reseult = citus_row("2020-01-01")



In [0]:

from pyspark.sql.types import *
def generate_citus_result(citus_data):
    schema = StructType([
    StructField("device_code", StringType(), True),
    StructField("sum_est_free_app_download", IntegerType(), True),
    StructField("sum_est_paid_app_download", IntegerType(), True),
    StructField("sum_est_revenue", IntegerType(), True)])

    df_3 = spark.createDataFrame(citus_reseult, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by device_code desc")

In [0]:

ios_feed={1:"0,1,2,100,101,102"}
print ios_feed.keys()[0] , ','.join(ios_feed.values())

In [0]:
   
ANDROID_CATEGORIES = [
    (1, 400000), (2, 400001), (3, 400022), (4, 400023), (5, 400024),
    (6, 400008), (7, 400011), (8, 400014), (9, 400017), (10, 400020),
    (11, 400025), (12, 400030), (13, 400031), (14, 400032), (15, 400033),
    (16, 400035), (17, 400036), (18, 400038), (19, 400040), (20, 400042),
    (21, 400043), (22, 400044), (23, 400058), (24, 400046), (25, 400047),
    (26, 400048), (27, 400050), (28, 400051), (29, 400052), (30, 400053),
    (31, 400054), (32, 400055), (33, 400056), (34, 400045), (35, 400057),
    (36, 400059), (37, 400060), (38, 400002), (39, 400003), (40, 400021),
    (41, 400004), (42, 400005), (43, 400006), (44, 400007), (46, 400009),
    (47, 400010), (48, 400012), (49, 400013), (51, 400015), (52, 400016),
    (54, 400018), (55, 400019), (56, 400061), (57, 400063), (58, 400064),
    (59, 400065), (60, 400062), (61, 400066), (62, 400067), (63, 400068),
    (64, 400069), (65, 400070), (66, 400026), (67, 400027), (68, 400041),
    (69, 400028), (70, 400029), (71, 400034), (72, 400037), (73, 400039),
    (75, 400049)
]


IOS_CATEGORIES = [
    (36, 100000), (100, 100021), (360, 100030), (361, 100031), (362, 100032),
    (363, 100033), (6000, 100023), (6001, 100077), (6002, 100076), (6003, 100075),
    (6004, 100073), (6005, 100072), (6006, 100070), (6007, 100069), (6008, 100068),
    (6009, 100067), (6010, 100066), (6011, 100065), (6012, 100034), (6013, 100029),
    (6014, 100001), (6015, 100027), (6016, 100026), (6017, 100025), (6018, 100022),
    (6020, 100064), (6021, 100035), (6022, 100024), (6023, 100028), (6024, 100071),
    (6025, 100074), (7001, 100002), (7002, 100003), (7003, 100004), (7004, 100005),
    (7005, 100006), (7006, 100007), (7007, 100008), (7008, 100009), (7009, 100010),
    (7010, 100011), (7011, 100012), (7012, 100013), (7013, 100014), (7014, 100015),
    (7015, 100016), (7016, 100017), (7017, 100018), (7018, 100019), (7019, 100020),
    (13001, 100053), (13002, 100046), (13003, 100049), (13004, 100054), (13005, 100060),
    (13006, 100037), (13007, 100036), (13008, 100038), (13009, 100039), (13010, 100040),
    (13011, 100041), (13012, 100042), (13013, 100043), (13014, 100044), (13015, 100045),
    (13017, 100047), (13018, 100048), (13019, 100050), (13020, 100051), (13021, 100052),
    (13023, 100055), (13024, 100056), (13025, 100057), (13026, 100058), (13027, 100059),
    (13028, 100061), (13029, 100062), (13030, 100063)
]


IOS_STORE_COUNTRY_MAPPING = [
    (0, 'WW'), (143575, 'AL'), (143563, 'DZ'), (143564, 'AO'), (143538, 'AI'),
    (143540, 'AG'), (143505, 'AR'), (143524, 'AM'), (143460, 'AU'), (143445, 'AT'),
    (143568, 'AZ'), (143539, 'BS'), (143559, 'BH'), (143541, 'BB'), (143565, 'BY'),
    (143446, 'BE'), (143555, 'BZ'), (143576, 'BJ'), (143542, 'BM'), (143577, 'BT'),
    (143556, 'BO'), (143525, 'BW'), (143503, 'BR'), (143543, 'VG'), (143560, 'BN'),
    (143526, 'BG'), (143578, 'BF'), (143579, 'KH'), (143455, 'CA'), (143580, 'CV'),
    (143544, 'KY'), (143581, 'TD'), (143483, 'CL'), (143465, 'CN'), (143501, 'CO'),
    (143582, 'CG'), (143495, 'CR'), (143494, 'HR'), (143557, 'CY'), (143489, 'CZ'),
    (143458, 'DK'), (143545, 'DM'), (143508, 'DO'), (143509, 'EC'), (143516, 'EG'),
    (143506, 'SV'), (143518, 'EE'), (143583, 'FJ'), (143447, 'FI'), (143442, 'FR'),
    (143584, 'GM'), (143443, 'DE'), (143573, 'GH'), (143448, 'GR'), (143546, 'GD'),
    (143504, 'GT'), (143585, 'GW'), (143553, 'GY'), (143510, 'HN'), (143463, 'HK'),
    (143482, 'HU'), (143558, 'IS'), (143467, 'IN'), (143476, 'ID'), (143449, 'IE'),
    (143491, 'IL'), (143450, 'IT'), (143511, 'JM'), (143462, 'JP'), (143528, 'JO'),
    (143517, 'KZ'), (143529, 'KE'), (143493, 'KW'), (143586, 'KG'), (143587, 'LA'),
    (143519, 'LV'), (143497, 'LB'), (143588, 'LR'), (143520, 'LT'), (143451, 'LU'),
    (143515, 'MO'), (143530, 'MK'), (143531, 'MG'), (143589, 'MW'), (143473, 'MY'),
    (143532, 'ML'), (143521, 'MT'), (143590, 'MR'), (143533, 'MU'), (143468, 'MX'),
    (143591, 'FM'), (143523, 'MD'), (143592, 'MN'), (143547, 'MS'), (143593, 'MZ'),
    (143594, 'NA'), (143484, 'NP'), (143452, 'NL'), (143461, 'NZ'), (143512, 'NI'),
    (143534, 'NE'), (143561, 'NG'), (143457, 'NO'), (143562, 'OM'), (143477, 'PK'),
    (143595, 'PW'), (143485, 'PA'), (143597, 'PG'), (143513, 'PY'), (143507, 'PE'),
    (143474, 'PH'), (143478, 'PL'), (143453, 'PT'), (143498, 'QA'), (143487, 'RO'),
    (143469, 'RU'), (143598, 'ST'), (143479, 'SA'), (143535, 'SN'), (143599, 'SC'),
    (143600, 'SL'), (143464, 'SG'), (143496, 'SK'), (143499, 'SI'), (143601, 'SB'),
    (143472, 'ZA'), (143466, 'KR'), (143454, 'ES'), (143486, 'LK'), (143548, 'KN'),
    (143549, 'LC'), (143550, 'VC'), (143554, 'SR'), (143602, 'SZ'), (143456, 'SE'),
    (143459, 'CH'), (143470, 'TW'), (143603, 'TJ'), (143572, 'TZ'), (143475, 'TH'),
    (143551, 'TT'), (143536, 'TN'), (143480, 'TR'), (143604, 'TM'), (143552, 'TC'),
    (143537, 'UG'), (143492, 'UA'), (143481, 'AE'), (143444, 'GB'), (143441, 'US'),
    (143514, 'UY'), (143566, 'UZ'), (143502, 'VE'), (143471, 'VN'), (143571, 'YE'),
    (143605, 'ZW')]
    
# IOS_STORE_COUNTRY_MAPPING = [
#     (0, 'WW'), (143441, 'US'), (143465, 'CN'), (143460, 'AU'), (143444, 'GB')
#     ]

ANDROID_STORE_COUNTRY_MAPPING = [
    (17, 'AR'), (1, 'AU'), (35, 'AT'), (61, 'AZ'), (11, 'BE'), (18, 'BR'), (47, 'BG'),
    (2, 'CA'), (13, 'CL'), (3, 'CN'), (52, 'CO'), (64, 'CR'), (80, 'HR'), (36, 'CZ'),
    (38, 'DK'), (62, 'EC'), (33, 'EG'), (20, 'FI'), (6, 'FR'), (4, 'DE'), (46, 'GR'),
    (16, 'HK'), (37, 'HU'), (19, 'IN'), (21, 'ID'), (39, 'IE'), (40, 'IL'), (8, 'IT'),
    (9, 'JP'), (53, 'KZ'), (95, 'KE'), (50, 'KW'), (86, 'LV'), (65, 'LB'), (78, 'LT'),
    (24, 'MY'), (26, 'MX'), (23, 'NL'), (41, 'NZ'), (74, 'NG'), (42, 'NO'), (54, 'PK'),
    (56, 'PE'), (31, 'PH'), (28, 'PL'), (43, 'PT'), (84, 'PR'), (73, 'QA'), (44, 'RO'),
    (22, 'RU'), (51, 'SA'), (32, 'SG'), (45, 'SK'), (14, 'ZA'), (27, 'KR'), (5, 'ES'),
    (34, 'SE'), (12, 'CH'), (30, 'TW'), (29, 'TH'), (25, 'TR'), (48, 'UA'), (49, 'AE'),
    (7, 'GB'), (10, 'US'), (15, 'VN'), (1000, 'WW')
]
# ANDROID_STORE_COUNTRY_MAPPING = [
#   (12, 'CH'), (30, 'TW'), (25, 'TR'), (48, 'UA'),
#     (7, 'GB'), (10, 'US'), (1000, 'WW')
# ]



def device_code_to_feed(market_code, device_code, metric_name):
    mapping = [
        ['apple-store',0,'ios-phone','est_free_app_download'],
        ['apple-store',1,'ios-phone','est_paid_app_download'],
        ['apple-store',2,'ios-phone','est_revenue'],
        ['apple-store',101,'ios-tablet','est_free_app_download'],
        ['apple-store',100,'ios-tablet','est_paid_app_download'],
        ['apple-store',102,'ios-tablet','est_revenue'],
        ['google-play',0,'android-all','est_free_app_download'],
        ['google-play',1,'android-all','est_paid_app_download'],
        ['google-play',2,'android-all','est_revenue'],
    ]
    return [x for x in mapping if (x[0], x[2], x[3]) == (market_code, device_code, metric_name)][0][1]



def country_code_to_id(market_code, code):
    if market_code == 'apple-store':
        ios_mapping = {_code:_id for (_id, _code) in IOS_STORE_COUNTRY_MAPPING}
        return ios_mapping[code]
    else:
        gp_mapping = {_code:_id for (_id, _code) in ANDROID_STORE_COUNTRY_MAPPING}
        return gp_mapping[code]


def category_to_legacy_category(market_code, legacy):
    if market_code == 'apple-store':
        ios_category = {_category:_legacy_category for (_legacy_category,_category) in IOS_CATEGORIES }
        return ios_category[legacy]
    else:
        gp_category =  {_category:_legacy_category for (_legacy_category,_category) in ANDROID_CATEGORIES }
        return gp_category[legacy]

country_code_to_id("google",'WW')
# 13028, 100061
# 71, 400034
# category_to_legacy_category("apple-store",100000)


In [0]:

store_id=country_code_to_id("apple-store","BY")
print store_id
feed_id=device_code_to_feed("apple-store","ios-tablet","est_free_app_download")
print feed_id
spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-03-01/").where("store_id={} and feed={}".format(store_id, feed_id)).show()
spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-03-01/").filter("country_code='BY' and device_code='ios-tablet'").show()


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row



def plproxy_row(date, country_code , market_code , device_code, metric_name):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )


    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }
    
    
    settings = build_db_settings(urn, "DAILY_EST")

    sql = '''
    select distinct app_id,'{}', '{}', date,estimate from plproxy.execute_select($$select
     distinct app_id,feed_id,store_id,estimate, date
    from aa.app_store_daily_estimate_0
        where 
      date = '{}' and store_id='{}' and feed_id = {}
        $$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT , date DATE)  '''.format(
           country_code,
           device_code,
        date, 
        country_code_to_id(market_code, country_code), 
        device_code_to_feed(market_code, device_code, metric_name)
    )

    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [ Row(app_id=r[0], country_code=r[1], device_code=r[2], date=r[3], estimate=r[4]) for r in rows ], len(rows)




In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row

def citus_row( date, country_code , market_code , device_code, metric_name):
    def get_data_in_citus(date, country_code , market_code , device_code, metric_name):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.10.254",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select app_id,country_code,device_code,date,{} as estimate from  store.store_est_fact_v1 where granularity='daily' and date='{}' and device_code='{}' and {} is not null and country_code='{}' ".format(metric_name, date, device_code, metric_name, country_code )
        db_data = query(citus_dsn_, sql)
        return db_data

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    
    result = get_data_in_citus(date, country_code , market_code , device_code, metric_name)
    return [ Row(app_id=r[0], country_code=r[1], device_code=r[2], date=r[3], estimate=r[4]) for r in result ], len(result)



# df_plploxy.createOrReplaceTempView("old")
# df_citus.createOrReplaceTempView("new")

# spark.sql("")



In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row

def plproxy_row_category(date, country_code , market_code , device_code, category_id,  metric_name):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )


    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }
    
    
    settings = build_db_settings(urn, "DAILY_EST")

    sql = '''
    select distinct app_id,'{}', '{}', '{}', date, estimate from plproxy.execute_select($$select
     distinct app_id,feed_id,store_id,estimate, date, category_id
    from aa.app_store_daily_estimate_1
        where 
      date = '{}' and store_id='{}' and feed_id = {} and category_id={}
        $$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT , date DATE, category_id INT)  '''.format(
           country_code,
           device_code,
           category_id,
           date, 
           country_code_to_id(market_code, country_code), 
           device_code_to_feed(market_code, device_code, metric_name),
           category_to_legacy_category(market_code,category_id)
        )

    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [ Row(app_id=r[0], country_code=r[1], device_code=r[2], category_id=[3], date=r[4], estimate=r[5]) for r in rows ], len(rows)

# plproxy_row_category('2020-01-02', 'US' , 'apple-store' , 'ios-phone', 100000,  'est_revenue')


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row

def citus_row_category(date, country_code , market_code , device_code, category_id, metric_name):
    def get_data_in_citus(date, country_code , market_code , device_code, category_id, metric_name):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.10.254",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select app_id,country_code,device_code,category_id, date,{} as estimate from  store.store_est_category_fact_v1 where granularity='daily' and date='{}' and device_code='{}' and {} is not null and country_code='{}' and category_id={} ".format(metric_name, date, device_code, metric_name, country_code, category_id )
        db_data = query(citus_dsn_, sql)
        return db_data

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    
    result = get_data_in_citus(date, country_code , market_code , device_code, category_id, metric_name)
    return [ Row(app_id=r[0], country_code=r[1], device_code=r[2], category_id=[3], date=r[4], estimate=r[5]) for r in result ] , len(result)

# citus_row_category('2020-03-28', 'US' , 'apple-store' , 'ios-phone', 100000,  'est_revenue')

In [0]:

est_list=["est_free_app_download", "est_paid_app_download", "est_revenue" ]
device_code_list=['ios-phone','ios-tablet']
date_list=['2019-11-11']
l1 = [[d, x[1], 'apple-store' , device_code, est] for x in IOS_STORE_COUNTRY_MAPPING for est in est_list for device_code in device_code_list for d in date_list]
l2 = [ [d, x[1], 'google-play' , "android-all", est] for x in ANDROID_STORE_COUNTRY_MAPPING for est in est_list for d in date_list]

test_data_list = l1 + l2

# print len(test_data_list)
# device_code_to_feed()
# def device_code_to_feed(market_code, device_code, metric_name):
print test_data_list


In [0]:

for test_data in test_data_list[1000:]:
    print test_data
    citus_rows, citus_db_size = citus_row(test_data[0],test_data[1],test_data[2],test_data[3],test_data[4]   )
    plproxy_rows , plproxy_db_size= plproxy_row(test_data[0],test_data[1],test_data[2],test_data[3],test_data[4]  )
    if plproxy_db_size != 0:
        spark.createDataFrame(plproxy_rows).createOrReplaceTempView("plproxy")
        try:
            spark.createDataFrame(citus_rows).createOrReplaceTempView("citus")
            except_count_1=spark.sql("select * from plproxy EXCEPT ALL select * from citus").count()
            db_count_diff=citus_db_size-plproxy_db_size
            if except_count_1 != 0:
                except_count_2=spark.sql("select * from citus EXCEPT ALL select * from plproxy").count()
                spark.createDataFrame([(test_data[0],test_data[1],test_data[2],test_data[3],test_data[4], except_count_1, except_count_2, db_count_diff)],
                schema=["date", "country_code" , "market_code" , "device_code", "est", "citus_plproxy" , "plproxy_citus" , "diff"]).write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/result_est_db_diff", mode="append")
                print 'failed',test_data[0],test_data[1],test_data[2],test_data[3],test_data[4], db_count_diff
        except Exception as exp:
            print exp
            print 'failed', test_data[0],test_data[1],test_data[2],test_data[3],test_data[4]
        



In [0]:

est_list=["est_free_app_download", "est_paid_app_download", "est_revenue" ]
device_code_list=['ios-phone','ios-tablet']
date_list=['2020-02-05']

l1 = [[d, x[1], 'apple-store' , device_code,category[1],est ] for x in IOS_STORE_COUNTRY_MAPPING for est in est_list for device_code in device_code_list for d in date_list for category in IOS_CATEGORIES ]
l2 = [ [d, x[1], 'google-play' , "android-all",category[1], est ] for x in ANDROID_STORE_COUNTRY_MAPPING for est in est_list for d in date_list for category in ANDROID_CATEGORIES ]

test_data_list_category = l1 + l2

print len(test_data_list_category)



In [0]:

for test_data in test_data_list_category[43100:]:
    plproxy_rows, plroxy_db_category_size = plproxy_row_category(test_data[0],test_data[1],test_data[2],test_data[3],test_data[4],test_data[5]  )
    citus_rows, citus_db_category_size = citus_row_category(test_data[0],test_data[1],test_data[2],test_data[3],test_data[4] ,test_data[5] )
    db_count_diff=citus_db_category_size-plroxy_db_category_size
    print test_data , 'diff is:' , db_count_diff
    if plroxy_db_category_size != 0:
        spark.createDataFrame(plproxy_rows).createOrReplaceTempView("plproxy_category")
        try:
            spark.createDataFrame(citus_rows).createOrReplaceTempView("citus_category")
            except_count_1=spark.sql("select * from citus_category EXCEPT ALL select * from plproxy_category").count()
            if except_count_1 != db_count_diff:
                except_count_2=spark.sql("select * from plproxy_category EXCEPT ALL select * from citus_category").count()
                spark.createDataFrame([(test_data[0],test_data[1],test_data[2],test_data[3],test_data[4], except_count_1, except_count_2, db_count_diff)],
                schema=["date", "country_code" , "market_code" , "device_code", "category_id",  "citus_plproxy" , "plproxy_citus" , "diff"]).write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/result_rank_db_diff", mode="append")

                print 'failed', test_data[0],test_data[1],test_data[2],test_data[3],test_data[4] ,test_data[5]
        except Exception as exp:
            print exp
            print 'failed', test_data[0],test_data[1],test_data[2],test_data[3],test_data[4],test_data[5]
        
        

In [0]:
%%sh



PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select sum(estimate) as cnt 
    from ( select app_id,feed_id,store_id,estimate,category_id,device_id,rank from aa.app_store_daily_estimate_1
    where date = '2018-04-01'  and  feed_id in (0) ) as prod
\$proxy\$) tbl (cnt bigint);

select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select sum(estimate) as cnt 
    from ( select app_id,feed_id,store_id,estimate,category_id,device_id,rank from aa.app_store_daily_estimate_1
    where date = '2018-04-01'  and  feed_id in (1) ) as prod
\$proxy\$) tbl (cnt bigint);


select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select sum(estimate) as cnt 
    from ( select app_id,feed_id,store_id,estimate,category_id,device_id,rank from aa.app_store_daily_estimate_1
    where date = '2018-04-01'  and  feed_id in (2) ) as prod
\$proxy\$) tbl (cnt bigint);


select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select sum(estimate) as cnt 
    from ( select app_id,feed_id,store_id,estimate,category_id,device_id,rank from aa.app_store_daily_estimate_1
    where date = '2018-04-01'  and  feed_id in (101) ) as prod
\$proxy\$) tbl (cnt bigint);

select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select sum(estimate) as cnt 
    from ( select app_id,feed_id,store_id,estimate,category_id,device_id,rank from aa.app_store_daily_estimate_1
    where date = '2018-04-01'  and  feed_id in (100) ) as prod
\$proxy\$) tbl (cnt bigint);


select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select sum(estimate) as cnt 
    from ( select app_id,feed_id,store_id,estimate,category_id,device_id,rank from aa.app_store_daily_estimate_1
    where date = '2018-04-01'  and  feed_id in (102)  ) as prod
\$proxy\$) tbl (cnt bigint);




select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select sum(estimate) as cnt 
    from ( select app_id,feed_id,store_id,estimate,category_id,device_id,rank from aa.app_store_daily_estimate_0
    where date = '2018-04-01'  and  feed_id in (0) ) as prod
\$proxy\$) tbl (cnt bigint);

select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select sum(estimate) as cnt 
    from ( select app_id,feed_id,store_id,estimate,category_id,device_id,rank from aa.app_store_daily_estimate_0
    where date = '2018-04-01'  and  feed_id in (1) ) as prod
\$proxy\$) tbl (cnt bigint);


select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$ 
    select sum(estimate) as cnt 
    from ( select app_id,feed_id,store_id,estimate,category_id,device_id,rank from aa.app_store_daily_estimate_0
    where date = '2018-04-01'  and  feed_id in (2) ) as prod
\$proxy\$) tbl (cnt bigint);


EOF
